In [3]:
import pandas as pd
from pathlib import Path

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"

In [4]:
this_file_dir = Path(".").resolve()
bld = this_file_dir / "bld"

In [5]:
bld = this_file_dir / "bld"
data_path = bld / "chs_data.dta"

In [4]:
raw = pd.read_stata(data_path)


In [33]:
def clean_and_reshape_nlsy_data(raw, info):
    """Create a list of cleaned yearly datasets and concatenates them into one DataFrame in long format.

    Args: 
        raw (DataFrame): The raw NLSY dataset.
        info (DataFrame): Information about variables in the dataset.

    Returns:
        Dataframe: A cleaned and reshaped DataFrame in long format with columns.
    """
    cleaned_data_list = []

    for year in range(1986, 2011, 2):
        cleaned_year_data = clean_year_data(raw, year, info)
        cleaned_data_list.append(cleaned_year_data)

    cleaned_and_reshaped_data = pd.concat(cleaned_data_list)
    
    return cleaned_and_reshaped_data   


def clean_year_data(raw, year, info): 
    df = pd.DataFrame(raw)
    df = df[info["nlsy_name"]]
    info_df = pd.DataFrame(info)
    df = _filter_year_data(df, year, info_df)
    for i in df.columns[3:]:
        df[i] = _clean_bpi_cat(df[i]) 
    df = _clean_raw_name(info_df, df)
    df = pd.concat([df, _add_subscale_scores(df)], axis=1)
    df["year"] = year
    df["childid"] = _change_data_types(df["childid"])
    df["momid"] = _change_data_types(df["momid"])
    df_complete = df.set_index(["childid", "year"])
    return df_complete

def _change_data_types(sr):
    return sr.astype(pd.UInt32Dtype())

def _add_subscale_scores(df):
    mapping_dict = {
    'not true': 0,
    'sometimes true': 1,
    'often true': 1
    }

    subscale = df.copy()
    for i in subscale.columns[2:]:
        subscale[i] = subscale[i].map(mapping_dict)
    
    categories = ["antisocial", "anxiety", "headstrong", "hyperactive", "dependence","peer"]

    for i in categories:
        subscale[i] = subscale[[col for col in subscale.columns if col.startswith(i)]].mean(axis=1)
    
    subscale = subscale[categories]
    return subscale


def _filter_year_data(raw, year, info_df):
    info_df_of_year = info_df.loc[ info_df['survey_year'].isin({str(year), "invariant"}) ]
    return raw[info_df_of_year["nlsy_name"]]


def _clean_raw_name(info_df, df):
    subinfo_df = info_df.loc[ info_df["nlsy_name"].isin(df.columns) ] 
    old_names = subinfo_df["nlsy_name"]
    new_names = subinfo_df["readable_name"]
    clean_name = dict(zip(old_names, new_names))
    df2 = df[old_names].rename(columns=clean_name)
    return df2


def _clean_bpi_cat(sr):
    sr = sr.replace([-7.0, -3.0, -2.0, -1.0], pd.NA)
    sr = sr.replace({'Never Attended School': pd.NA, 'Multiple selection': pd.NA })
    categories = ["not true", "sometimes true", "often true"]
    sr = sr.astype(pd.StringDtype()).str.lower().astype(pd.CategoricalDtype(categories=categories, ordered=True))
    return sr


In [39]:
raw = pd.read_stata(bld / "BEHAVIOR_PROBLEMS_INDEX.dta")
info = pd.read_csv(bld / "bpi_variable_info.csv")
chs = pd.read_stata(bld / "chs_data.dta")

In [40]:
chs.head()

,childid,age,momid,birthorder,year,race,sex,yob,nchildren,momage,...,dfamincB,dmomhgc,dlogsal30,dconvict,dprobat,ddrug1,ddrug2,drepeat,dspecial,dteenpreg
0,201.0,0,2.0,1,1993,3.0,1.0,1993.0,2.0,34.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,201.0,1,2.0,1,1994,3.0,1.0,1993.0,2.0,34.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,201.0,3,2.0,1,1996,3.0,1.0,1993.0,2.0,34.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,201.0,5,2.0,1,1998,3.0,1.0,1993.0,2.0,34.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,201.0,7,2.0,1,2000,3.0,1.0,1993.0,2.0,34.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [37]:
_filter_year_data(raw,1998,info)

,C0000100,C0000200,C0005800,C1977000,C1977100,C1977200,C1977300,C1977400,C1977500,C1977600,...,C1979200,C1979300,C1979400,C1979500,C1979600,C1979700,C1979800,C1979900,C1980000,C1980100
0,201.0,2.0,1,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,...,NOT TRUE,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,Not true,Not true
1,202.0,2.0,2,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
2,301.0,3.0,1,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
3,302.0,3.0,2,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
4,303.0,3.0,3,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,SOMETIMES TRUE,...,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,Not true,Not true
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11516,1267201.0,12672.0,1,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
11517,1267202.0,12672.0,2,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
11518,1267301.0,12673.0,1,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
11519,1267302.0,12673.0,2,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0


In [35]:
df = clean_year_data(raw,1998,info)
df.loc[201]['anxiety_sad']

year
1998    not true
Name: anxiety_sad, dtype: category
Categories (3, string): [not true < sometimes true < often true]

In [32]:
df = clean_year_data(raw,1998,info)

print([df.loc[201][i] for i in df.columns if i.startswith('anxiety')])

[year
1998    not true
Name: anxiety_mood, dtype: category
Categories (3, string): [not true < sometimes true < often true], year
1998    not true
Name: anxiety_complain, dtype: category
Categories (3, string): [not true < sometimes true < often true], year
1998    sometimes true
Name: anxiety_fearful, dtype: category
Categories (3, string): [not true < sometimes true < often true], year
1998    not true
Name: anxiety_worthless, dtype: category
Categories (3, string): [not true < sometimes true < often true], year
1998    not true
Name: anxiety_sad, dtype: category
Categories (3, string): [not true < sometimes true < often true], year
1998    0.2
Name: anxiety, dtype: float64]


In [57]:
def clean_year_data(raw,year, info):
    df = pd.DataFrame(index=raw.index)
    df = _clean_bpi_variables(raw,info)
    for i in df.columns[:2]:
        df[i]= df[i].astype(pd.Int32Dtype()).astype(pd.CategoricalDtype())
    for i in df.columns[3:]:
        df[i] = _clean_bpi_cat(df[i])
    
    df = pd.wide_to_long(df, stubnames= info.readable_name.unique(),i= 'childid invariant', j='year', sep= " ")
    df = df.drop(columns=['childid','momid','birth_order'])
    df.index.names = ['childid', 'year']
    df = df.rename(columns= {'momid invariant': 'momid', 'birth_order invariant': 'birth_order'})
    df = df.sort_index()
    df = pd.concat([df, _add_subscale_scores(df)], axis=1)
    return df.xs(year, level=1)

def _clean_bpi_variables(raw_df, info_df):
     raw_df = raw_df[info_df["nlsy_name"]]  # choosing variables that are available in info 
     clean_variables = dict(zip(info_df.nlsy_name, info_df.readable_name +  ' ' + info_df.survey_year)) # creating a dictionary to rename columns in raw data
     return raw_df.rename(columns=clean_variables)

def _clean_bpi_cat(sr):
    sr = sr.replace([-7.0, -3.0, -2.0, -1.0], pd.NA)
    sr = sr.replace({'Never Attended School': pd.NA, 'Multiple selection': pd.NA })
    categories = ["not true", "sometimes true", "often true"]
    sr = sr.astype(pd.StringDtype()).str.lower().astype(pd.CategoricalDtype(categories=categories, ordered=True))
    return sr

def _add_subscale_scores(df):
    mapping_dict = {
    'not true': 0,
    'sometimes true': 1,
    'often true': 1
    }
    subscale = df.copy()
    for i in subscale.columns[2:]:
        subscale[i] = subscale[i].map(mapping_dict)
    
    categories = ["antisocial", "anxiety", "headstrong", "hyperactive", "dependence","peer"]

    for i in categories:
        subscale[i] = subscale[[col for col in subscale.columns if col.startswith(i)]].mean(axis=1)
    
    subscale = subscale[categories]

    return subscale

In [58]:
df = clean_year_data(raw,info)

TypeError: clean_year_data() missing 1 required positional argument: 'info'

In [ ]:
clean_year_data(raw, 1998, info)

,,momid,birth_order,antisocial,anxiety,headstrong,hyperactive,dependence,peer
childid,year,,,,,,,,
201.0,1998,2.0,1,NaN,NaN,NaN,NaN,NaN,NaN
202.0,1998,2.0,2,NaN,NaN,NaN,NaN,NaN,NaN
301.0,1998,3.0,1,NaN,NaN,NaN,NaN,NaN,NaN
302.0,1998,3.0,2,NaN,NaN,NaN,NaN,NaN,NaN
303.0,1998,3.0,3,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1267201.0,1998,12672.0,1,NaN,NaN,NaN,NaN,NaN,NaN
1267202.0,1998,12672.0,2,NaN,NaN,NaN,NaN,NaN,NaN
1267301.0,1998,12673.0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df.loc[201: 302]

momid birth_order    anxiety_mood anxiety_complain  \
childid year                                                      
201     1986     2           1             NaN              NaN   
        1988     2           1             NaN              NaN   
        1990     2           1             NaN              NaN   
        1992     2           1             NaN              NaN   
        1994     2           1             NaN              NaN   
        1996     2           1             NaN              NaN   
        1998     2           1        not true         not true   
        2000     2           1        not true   sometimes true   
        2002     2           1        not true         not true   
        2004     2           1        not true         not true   
        2006     2           1        not true         not true   
        2008     2           1             NaN              NaN   
        2010     2           1             NaN              NaN   
202     1986     2           2             NaN              NaN   
        1988     2           2             NaN              NaN   
        1990     2           2             NaN              NaN   
        1992     2           2             NaN              NaN   
        1994     2           2             NaN              NaN   
        1996     2           2             NaN              NaN   
        1998     2           2             NaN              NaN   
        2000     2           2        not true   sometimes true   
        2002     2           2        not true         not true   
        2004     2           2        not true         not true   
        2006     2           2        not true         not true   
        2008     2           2        not true         not true   
        2010     2           2             NaN              NaN   
301     1986     3           1        not true         not true   
        1988     3           1        not true   sometimes true   
        1990     3           1        not true   sometimes true   
        1992     3           1  sometimes true   sometimes true   
        1994     3           1  sometimes true   sometimes true   
        1996     3           1             NaN              NaN   
        1998     3           1             NaN              NaN   
        2000     3           1             NaN              NaN   
        2002     3           1             NaN              NaN   
        2004     3           1             NaN              NaN   
        2006     3           1             NaN              NaN   
        2008     3           1             NaN              NaN   
        2010     3           1             NaN              NaN   
302     1986     3           2             NaN              NaN   
        1988     3           2        not true         not true   
        1990     3           2      often true       often true   
        1992     3           2  sometimes true   sometimes true   
        1994     3           2  sometimes true   sometimes true   
        1996     3           2             NaN              NaN   
        1998     3           2             NaN              NaN   
        2000     3           2             NaN              NaN   
        2002     3           2             NaN              NaN   
        2004     3           2             NaN              NaN   
        2006     3           2             NaN              NaN   
        2008     3           2             NaN              NaN   
        2010     3           2             NaN              NaN   

             headstrong_tense antisocial_cheat anxiety_fearful  \
childid year                                                     
201     1986              NaN              NaN             NaN   
        1988              NaN              NaN             NaN   
        1990              NaN              NaN             NaN   
        1992              NaN              NaN             NaN  

In [ ]:
yearly_data = [clean_year_data(raw, i, info) for i in range(1986, 2011, 2)]


KeyboardInterrupt: 

In [ ]:
raw.head()

,C0000100,C0000200,C0005800,C0564000,C0564100,C0564200,C0564300,C0564400,C0564500,C0564600,...,Y3249000,Y3249001,Y3249100,Y3249101,Y3249200,Y3249201,Y3249300,Y3249301,Y3249400,Y3249401
0,201.0,2.0,1,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
1,202.0,2.0,2,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
2,301.0,3.0,1,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
3,302.0,3.0,2,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0
4,303.0,3.0,3,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0,...,Not True,Not True,Not True,Not True,-7.0,-7.0,-7.0,-7.0,-7.0,-7.0


In [ ]:
info.head()

,nlsy_name,readable_name,survey_year,label
0,C0000100,childid,invariant,id code of child
1,C0000200,momid,invariant,id code of mother of child
2,C0005800,birth_order,invariant,birth order of child
3,C0564000,anxiety_mood,1986,ch has sud chgs in mood/feelng
4,C0564100,anxiety_complain,1986,ch cmplns no one loves him/her


In [ ]:
info = info.set_index("nlsy_name")

,readable_name,survey_year,label
nlsy_name,,,
C0000100,childid,invariant,id code of child
C0000200,momid,invariant,id code of mother of child
C0005800,birth_order,invariant,birth order of child
C0564000,anxiety_mood,1986,ch has sud chgs in mood/feelng
C0564100,anxiety_complain,1986,ch cmplns no one loves him/her
...,...,...,...
C5192400,additional_hangout,2010,hang arnd kids who get in trbl
C5192500,additional_secretive,2010,child is secretive
C5192600,additional_worries,2010,child worries too much


In [ ]:
info.index.name = None

In [ ]:
info

,readable_name,survey_year,label
C0000100,childid,invariant,id code of child
C0000200,momid,invariant,id code of mother of child
C0005800,birth_order,invariant,birth order of child
C0564000,anxiety_mood,1986,ch has sud chgs in mood/feelng
C0564100,anxiety_complain,1986,ch cmplns no one loves him/her
...,...,...,...
C5192400,additional_hangout,2010,hang arnd kids who get in trbl
C5192500,additional_secretive,2010,child is secretive
C5192600,additional_worries,2010,child worries too much
C5192800,antisocial_disob_school,2010,ch is disobedient at school


In [ ]:
import pandas as pd
from pathlib import Path

pd.options.mode.copy_on_write = True
pd.options.future.infer_string = True
pd.options.plotting.backend = "plotly"

this_file_dir = Path(__name__).parent # if it doesn't work - try this_file_dir = Path(__file__).parent
bld = this_file_dir / "bld"
data_path = bld / "BEHAVIOR_PROBLEMS_INDEX.dta"

In [59]:
def clean_year_data(raw, year, info): 
    df = pd.DataFrame(raw)
    df = df[info["nlsy_name"]]
    info_df = pd.DataFrame(info)
    df = _filter_year_data(df, year, info_df)
    bpi = df.columns[3:]
    df[bpi] = _clean_check_surprises(df[bpi])
    df[bpi] = _clean_bpi_info(df[bpi]) 
    df = _clean_raw_name(info_df, df)
    df = pd.concat([df, _add_subscale_scores(df)], axis=1)
    df["year"] = year
    df["childid"] = _change_data_types(df["childid"])
    df["momid"] = _change_data_types(df["momid"])
    df_complete = df.set_index(["childid", "year"])
    return df_complete

def _change_data_types(sr):
    return sr.astype(pd.UInt32Dtype())

def _add_subscale_scores(df):
    mapping_dict = {
    'NOT TRUE': 0,
    'SOMETIMES TRUE': 1,
    'OFTEN TRUE': 1
    }

    subscale = df.copy()
    for i in subscale.columns[2:]:
        subscale[i] = subscale[i].map(mapping_dict)
    
    categories = ["antisocial", "anxiety", "headstrong", "hyperactive", "dependence","peer"]

    for i in categories:
        subscale[i] = subscale[[col for col in subscale.columns if col.startswith(i)]].mean(axis=1)
    
    subscale = subscale[categories]

    return subscale


def _filter_year_data(raw, year, info_df):
    info_df_of_year = info_df.loc[ info_df['survey_year'].isin({str(year), "invariant"}) ]
    return raw[info_df_of_year["nlsy_name"]]


def _clean_raw_name(info_df, df):
    subinfo_df = info_df.loc[ info_df["nlsy_name"].isin(df.columns) ] 
    old_names = subinfo_df["nlsy_name"]
    new_names = subinfo_df["readable_name"]
    clean_name = dict(zip(old_names, new_names))
    df2 = df[old_names].rename(columns=clean_name)
    return df2


def _clean_bpi_info(sr):
    sr = sr.replace([-7.0, -2.0, -1.0], pd.NA).replace(["Never Attended School", "Multiple selections"], pd.NA)    
    categories = ["NOT TRUE", "SOMETIMES TRUE", "OFTEN TRUE"]
    dtype = pd.CategoricalDtype(categories=categories, ordered=True)
    return sr.astype(dtype)


def _clean_check_surprises(sr):
    for col in sr.columns:
        sr[col].str.upper().str.strip()
    return sr.astype(pd.CategoricalDtype())


In [60]:
raw = pd.read_stata(bld / "BEHAVIOR_PROBLEMS_INDEX.dta")
info = pd.read_csv(bld / "bpi_variable_info.csv")

In [61]:
clean_year_data(raw,1998,info)

,,momid,birth_order,anxiety_mood,anxiety_complain,headstrong_tense,antisocial_cheat,anxiety_fearful,headstrong_argues,hyperactive_concentration,hyperactive_confused,...,additional_secretive,additional_worries,antisocial_disob_school,antisocial_teachers,antisocial,anxiety,headstrong,hyperactive,dependence,peer
childid,year,,,,,,,,,,,,,,,,,,,,,
201,1998,2,1,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,...,NOT TRUE,SOMETIMES TRUE,NaN,NaN,0.0,0.2,0.2,0.4,NaN,0.000000
202,1998,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
301,1998,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
302,1998,3,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,1998,3,3,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,SOMETIMES TRUE,NOT TRUE,...,NOT TRUE,NOT TRUE,NaN,NaN,0.0,0.4,0.8,0.2,NaN,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1267201,1998,12672,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267202,1998,12672,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1267301,1998,12673,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
def clean_and_reshape_nlsy_data(raw, info):
    yearly_data = []
    for i in range(1986,2011, 2):
        yearly_data.append(clean_year_data(raw, i, info))
  
    df = pd.concat(yearly_data)
    return df

df_1 = clean_and_reshape_nlsy_data(raw,info)
df_1 = df_1.sort_index(axis=0)
    

In [63]:
df_1

momid birth_order anxiety_mood anxiety_complain  \
childid year                                                    
201     1986      2           1          NaN              NaN   
        1988      2           1          NaN              NaN   
        1990      2           1          NaN              NaN   
        1992      2           1          NaN              NaN   
        1994      2           1          NaN              NaN   
...             ...         ...          ...              ...   
1267501 2002  12675           1          NaN              NaN   
        2004  12675           1          NaN              NaN   
        2006  12675           1          NaN              NaN   
        2008  12675           1          NaN              NaN   
        2010  12675           1          NaN              NaN   

             headstrong_tense antisocial_cheat anxiety_fearful  \
childid year                                                     
201     1986              NaN              NaN             NaN   
        1988              NaN              NaN             NaN   
        1990              NaN              NaN             NaN   
        1992              NaN              NaN             NaN   
        1994              NaN              NaN             NaN   
...                       ...              ...             ...   
1267501 2002              NaN              NaN             NaN   
        2004              NaN              NaN             NaN   
        2006              NaN              NaN             NaN   
        2008              NaN              NaN             NaN   
        2010              NaN              NaN             NaN   

             headstrong_argues hyperactive_concentration hyperactive_confused  \
childid year                                                                    
201     1986               NaN                       NaN                  NaN   
        1988               NaN                       NaN                  NaN   
        1990               NaN                       NaN                  NaN   
        1992               NaN                       NaN                  NaN   
        1994               NaN                       NaN                  NaN   
...                        ...                       ...                  ...   
1267501 2002               NaN                       NaN                  NaN   
        2004               NaN                       NaN                  NaN   
        2006               NaN                       NaN                  NaN   
        2008               NaN                       NaN                  NaN   
        2010               NaN                       NaN                  NaN   

              ... antisocial anxiety headstrong hyperactive dependence peer  \
childid year  ...                                                             
201     1986  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        1988  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        1990  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        1992  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        1994  ...        NaN     NaN        NaN         NaN        NaN  NaN   
...           ...        ...     ...        ...         ...        ...  ...   
1267501 2002  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        2004  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        2006  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        2008  ...        NaN     NaN        NaN         NaN        NaN  NaN   
        2010  ...        NaN     NaN        NaN         NaN        NaN  NaN   

             additional_gethim additional_hangout additional_secretive  \
childid year                                                             
201     1986               NaN                NaN                  NaN   
     

In [64]:
df_new = pd.merge(df_1,df,on=['childid','year'], how='left')

In [40]:
df_new.loc[301]

,momid_x,birth_order,anxiety_mood,anxiety_complain,headstrong_tense,antisocial_cheat,anxiety_fearful,headstrong_argues,hyperactive_concentration,hyperactive_confused,...,additional_hangout,additional_secretive,additional_worries,bpiA,bpiB,bpiC,bpiD,bpiE,momid_y,age
year,,,,,,,,,,,,,,,,,,,,,
1986,3,1,NOT TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,...,NaN,NaN,NaN,0.954977,0.252469,1.374069,0.561555,-0.852121,3,5
1988,3,1,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,SOMETIMES TRUE,NOT TRUE,...,NOT TRUE,NOT TRUE,SOMETIMES TRUE,0.940177,-0.305677,0.165581,-0.696726,0.57343,3,7
1990,3,1,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,SOMETIMES TRUE,SOMETIMES TRUE,...,NOT TRUE,SOMETIMES TRUE,SOMETIMES TRUE,0.249955,-0.212118,-0.422873,-0.089564,0.59118,3,9
1992,3,1,SOMETIMES TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,...,NOT TRUE,NOT TRUE,SOMETIMES TRUE,0.244763,-0.193752,0.181487,0.473365,0.597735,3,11
1994,3,1,SOMETIMES TRUE,SOMETIMES TRUE,NOT TRUE,NOT TRUE,NOT TRUE,SOMETIMES TRUE,OFTEN TRUE,SOMETIMES TRUE,...,NOT TRUE,NOT TRUE,NOT TRUE,<NA>,-0.19158,0.212737,-0.209371,0.584663,3,13
1996,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>
1998,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>
2000,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>
2002,3,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>
